# <center>  **<span style="font-size:80px;">Graph Matching</span>** </center>

# <center> Universidad de Alicante </center>
<p align="center"><img src=".ipynb_gfx/UA.png" width="330" height="190"></p>
<br>

**Nombre:** Santiago Álvarez Geanta <br>
**Fecha:** 7 de febrero de 2025 <br>
**Grado:** Ingeniería en Inteligencia Artificial <br>
**Grupo:** 1 <br>


### Profesores
- Francisco Escolano Ruíz
- Ahmed Begga Hachlafi
<br>

# 1. Ejercicios

## 1.1 Ejercicio 1

A continuación vamos a implementar el Algoritmo Hungaro para el Graph Matching. Nuestro objetivo será encontrar la asignación/matcheo óptima de puntos entre dos imágenes. Usaremos la matriz de costes creada a partir de las distancias Uclídeas entre los puntos. Para resolver el problema utilizaremos la función `scipy.optimize.linear_sum_assignment`.

In [4]:
import numpy as np

In [5]:
import numpy as np
from PIL import Image
import scipy.io as sio
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay 
from scipy.optimize import linear_sum_assignment


def load_and_preprocess_images(img_path1, img_path2, mat_path1, mat_path2):
    """
    Carga y preprocesa pares de imágenes y sus keypoints
    """
    return img1, img2, kpts1, kpts2
def delaunay_triangulation(kpt):
    """
    Generate adjacency matrix based on Delaunay triangulation
    """
    return A

def simple_spatial_matching(kpts1, kpts2):
    """
    Realiza matching entre puntos usando distancia euclidiana y algoritmo húngaro optimizado
    
    Args:
        kpts1: Array de keypoints del primer grafo (2xN)
        kpts2: Array de keypoints del segundo grafo (2xN)
    
    Returns:
        matching: Matriz binaria donde matching[i,j]=1 indica correspondencia entre puntos
    Hints:
        - Primero deberás crear una matriz de costes basada en distancias euclidianas,
        cuyo tamaño será (n1, n2) donde n1 y n2 son el número de keypoints en cada grafo.
        - Seguidamente, deberás rellenar dicha matriz con las distancias euclidianas entre
        los keypoints de ambos grafos.
        - A continuación, deberás aplicar el algoritmo húngaro usando la función linear_sum_assignment, que recibe la matriz de costes y devuelve los índices de los puntos emparejados.
        - Finalmente, deberás crear una matriz de matching a partir de los índices obtenidos.

    """
    return matching

def visualize_matching_full(img1, img2, kpts1, kpts2, adj_matrix1, adj_matrix2, matching):
    """
    Visualiza el matching completo entre dos grafos
    """

In [6]:
def HungarianAlgorithm(cost_matrix: np.array):
    cost_matrix = cost_matrix.copy()
    num_rows, num_cols = cost_matrix.shape
    
    ##########################
    # Nos aseguramos de que en cada fila y columna haya
    #  al menos un 0 para encontrar los valores más próximos

    # Paso 1: Reducción de filas
    for i in range(num_rows):
        cost_matrix[i] -= cost_matrix[i].min()

    # Paso 2: Reducción de columnas
    for j in range(num_cols):
        cost_matrix[:, j] -= cost_matrix[:, j].min()

    # Paso 3 y 4: Cubrir los ceros con el número mínimo de líneas
    while True:
        row_cover = np.zeros(num_rows, dtype=bool)
        col_cover = np.zeros(num_cols, dtype=bool)
        marked_zeros = np.zeros_like(cost_matrix, dtype=bool)

        for i in range(num_rows):
            for j in range(num_cols):
                if cost_matrix[i, j] == 0 and not row_cover[i] and not col_cover[j]:
                    marked_zeros[i, j] = True
                    row_cover[i] = True
                    col_cover[j] = True
        
        num_lines = np.sum(row_cover) + np.sum(col_cover)

        if num_lines >= num_rows:
            break  # Se encontró una asignación óptima

        # Paso 5: Ajuste de la matriz
        uncovered_vals = cost_matrix[~row_cover, :][:, ~col_cover]
        min_uncovered = uncovered_vals.min()

        # Restar min_uncovered de los valores no cubiertos y sumarlo a los valores cubiertos dos veces
        cost_matrix[~row_cover, :] -= min_uncovered
        cost_matrix[:, col_cover] += min_uncovered

    # Paso 6: Encontrar la asignación óptima
    assignment = np.zeros_like(cost_matrix, dtype=int)
    for i in range(num_rows):
        for j in range(num_cols):
            if marked_zeros[i, j]:
                assignment[i, j] = 1

    return assignment

if __name__ == "__main__":
    # Ejemplo de uso
    cost_matrix = np.array([
        [4, 2, 8],
        [4, 3, 7],
        [6, 4, 7]
    ])

    optimal_assignment = HungarianAlgorithm(cost_matrix)
    print("Asignación óptima:\n", optimal_assignment)


Asignación óptima:
 [[0 1 0]
 [1 0 0]
 [0 0 1]]


### <center>Santiago Álvarez Geanta</center>
#### <center>Universidad de Alicante</center>
<p align="center"><img src=".ipynb_gfx/UA.png" width="165" height="95"></p>